# Introduction to Snowpark Connect for Apache Spark

### What You'll Learn:
- How Snowpark Connect executes PySpark on Snowflake infrastructure
- Data ingestion patterns (tables, stages, cloud storage)
- Transformations, joins, and aggregations
- Writing data with partitioning and compression
- Building production pipelines with telemetry

## What is Snowpark Connect?

Snowpark Connect allows you to run the **PySpark DataFrame API** on **Snowflake infrastructure**.

In [ ]:
# import packages
from snowflake import snowpark_connect
from snowflake.snowpark.context import get_active_session

# initialize session
session = get_active_session()
spark = snowpark_connect.server.init_spark_session()

# print session info
print(session)

### Key Concepts:

**Execution Model:**
- Your DataFrame operations are translated to Snowflake SQL
- Computation happens in Snowflake warehouses
- Results stream back via Apache Arrow format
- No Spark cluster, driver, or executors

**Query Pushdown:**
- ✅ **Fully Optimized:** DataFrame operations, SQL functions, aggregations push down to Snowflake
- ⚠️ **Performance Impact:** Python UDFs run client-side (fetch data → process → send back)
- 💡 **Better Alternative:** Use built-in SQL functions instead of UDFs

## Data Ingestion

In [ ]:
# =============================================================================
# DATA INGESTION METHODS
# =============================================================================

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, BooleanType

# =============================================================================

# -----------------------------------------------------------------------------
# 1. READ FROM SNOWFLAKE TABLES
# -----------------------------------------------------------------------------

# Read table by name
#   - Fastest method - data is already in Snowflake, no file parsing
df = spark.read.table("MY_DATABASE.MY_SCHEMA.MY_TABLE")

# Execute SQL query and return results
#   - Useful for filtering at read time to reduce data transfer
#   - Supports all Snowflake SQL syntax
df = spark.sql("SELECT * FROM MY_TABLE WHERE status = 'active' LIMIT 1000")

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 2. READ FROM SNOWFLAKE STAGES
# -----------------------------------------------------------------------------

# PARQUET:
df = spark.read.parquet("@MY_STAGE/path/to/data.parquet")

# CSV:
df = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", False) \
    .option("delimiter", ",") \
    .option("quote", '"') \
    .option("escape", "\\") \
    .option("nullValue", "") \
    .option("dateFormat", "yyyy-MM-dd") \
    .load("@MY_STAGE/path/to/data.csv")

# JSON:
df = spark.read.format("json") \
    .option("multiLine", True) \
    .option("allowComments", True) \
    .option("dateFormat", "yyyy-MM-dd") \
    .load("@MY_STAGE/path/to/data.json")

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 3. DIRECT CLOUD STORAGE ACCESS
# -----------------------------------------------------------------------------

# Read from S3
#   - Requires AWS credentials configured
df = spark.read.parquet("s3://my-bucket/path/to/data/")

# Read from Google Cloud Storage
#   - Requires GCP credentials configured
df = spark.read.parquet("gs://my-bucket/path/to/data/")

# Read from Azure Blob Storage
#   - Requires Azure credentials configured
df = spark.read.parquet("wasbs://container@account.blob.core.windows.net/path/to/data/")

# NOTE: Using Snowflake stages is often simpler (handles auth automatically)

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 4. READ MULTIPLE FILES
# -----------------------------------------------------------------------------

# Read with wildcard patterns
#   - Match all files of a type in a directory
df = spark.read.parquet("@MY_STAGE/data/*.parquet")
#   - Match files with a naming pattern
df = spark.read.csv("@MY_STAGE/logs/2024-*/events_*.csv")

# Read recursively from nested directories
#   - Searches all subdirectories
df = spark.read.option("recursiveFileLookup", True).parquet("@MY_STAGE/nested_data/")

# -----------------------------------------------------------------------------

# =============================================================================
# HANDLING LARGE CSVs
# ============================================================================

# ❌ SLOW PATTERN:
df_slow = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("@MY_STAGE/large_file.csv")
#    Problem: Scans entire file just to guess column types

# ✅ FAST PATTERN:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("created_at", TimestampType(), True),
    StructField("is_active", BooleanType(), True)
])

df_fast = spark.read.format("csv") \
    .option("header", True) \
    .schema(schema) \
    .load("@MY_STAGE/large_file.csv")
#    Benefit: No extra scan

# ✅ COMPRESSED FILES:
df_gzip = spark.read.format("csv") \
    .option("header", True) \
    .schema(schema) \
    .load("@MY_STAGE/data.csv.gz")
#    Benefit: Smaller transfer size, auto-decompressed on read

# ✅ BEST PRACTICE - Convert to Parquet:
df_csv = spark.read.format("csv").option("header", True).schema(schema).load("@MY_STAGE/raw.csv")
df_csv.write.mode("overwrite").parquet("@MY_STAGE/optimized/data.parquet")
df_optimized = spark.read.parquet("@MY_STAGE/optimized/data.parquet")
#    Benefit: Fast columnar reads for repeated analysis

# =============================================================================


## Writing Data

In [ ]:
# =============================================================================
# WRITING DATA
# =============================================================================
#
# Write transformed data back to Snowflake tables, stages, or cloud storage
#
# =============================================================================

# -----------------------------------------------------------------------------
# 1. WRITE TO SNOWFLAKE TABLES
# -----------------------------------------------------------------------------

# Save DataFrame as a Snowflake table
df.write.mode("overwrite").saveAsTable("MY_TABLE")
df.write.mode("append").saveAsTable("MY_TABLE")
df.write.mode("ignore").saveAsTable("MY_TABLE")
df.write.mode("error").saveAsTable("MY_TABLE")  # default behavior

# Write to fully qualified table name
df.write.mode("overwrite").saveAsTable("MY_DATABASE.MY_SCHEMA.MY_TABLE")

# NOTE: This is the fastest write path - data stays in Snowflake

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 2. WRITE TO SNOWFLAKE STAGES
# -----------------------------------------------------------------------------

# PARQUET (recommended):
df.write.mode("overwrite").parquet("@MY_STAGE/output/data.parquet")
#   - Best for analytical workloads
#   - Columnar format, compressed by default
#   - Preserves schema information

# CSV:
df.write.mode("overwrite") \
    .option("header", True) \
    .option("compression", "gzip") \
    .option("delimiter", ",") \
    .option("quote", '"') \
    .csv("@MY_STAGE/output/data.csv")

# JSON:
df.write.mode("overwrite") \
    .option("compression", "gzip") \
    .json("@MY_STAGE/output/data.json")
#   - One JSON object per row

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 3. WRITE TO CLOUD STORAGE
# -----------------------------------------------------------------------------

# Write to S3
df.write.mode("overwrite").parquet("s3://my-bucket/output/data/")

# Write to Google Cloud Storage
df.write.mode("overwrite").parquet("gs://my-bucket/output/data/")

# Write to Azure Blob Storage
df.write.mode("overwrite").parquet("wasbs://container@account.blob.core.windows.net/output/data/")

# NOTE: Using Snowflake stages is often simpler (handles auth automatically)

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 4. PARTITIONING FOR PERFORMANCE
# -----------------------------------------------------------------------------

# Write with partitioning by columns
df.write.mode("overwrite") \
    .partitionBy("year", "month") \
    .parquet("@MY_STAGE/partitioned_data/")

# Single column partition (common for date-based)
df.write.mode("overwrite") \
    .partitionBy("event_date") \
    .parquet("@MY_STAGE/daily_data/")

# Benefits:
#   - Faster queries that filter on partition columns
#   - Enables parallel reads of different partitions
#   - Easier data management (delete old partitions)

# Considerations:
#   - Too many partitions = too many small files
#   - Choose columns with moderate cardinality
#   - Date-based partitioning is very common

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 5. COMPRESSION OPTIONS
# -----------------------------------------------------------------------------

# Snappy (default for Parquet) - fast, moderate compression
df.write.option("compression", "snappy").parquet("@MY_STAGE/snappy_data/")

# Gzip - slower, better compression
df.write.option("compression", "gzip").parquet("@MY_STAGE/gzip_data/")

# LZ4 - very fast, moderate compression
df.write.option("compression", "lz4").parquet("@MY_STAGE/lz4_data/")

# Zstd - good balance of speed and compression
df.write.option("compression", "zstd").parquet("@MY_STAGE/zstd_data/")

# None - no compression (rarely recommended)
df.write.option("compression", "none").parquet("@MY_STAGE/uncompressed_data/")

# Trade-offs:
#   - Higher compression = smaller files but slower writes
#   - Snappy/LZ4 good for frequently accessed data
#   - Gzip/Zstd good for archival or infrequent access

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 6. CONTROLLING OUTPUT FILE COUNT
# -----------------------------------------------------------------------------

# Reduce number of output files (coalesce)
df.coalesce(1).write.mode("overwrite").parquet("@MY_STAGE/single_file/")   # Single file
df.coalesce(4).write.mode("overwrite").parquet("@MY_STAGE/few_files/")     # 4 files max

# Increase number of output files (repartition)
df.repartition(100).write.mode("overwrite").parquet("@MY_STAGE/many_files/")

# Repartition by column (ensures related data in same files)
df.repartition("customer_id").write.mode("overwrite").parquet("@MY_STAGE/by_customer/")

# Best practices:
#   - Target 100MB-1GB per file for optimal read performance
#   - Too many small files = slow reads (file listing overhead)
#   - Too few large files = limited parallelism

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 7. WRITE MODES SUMMARY
# -----------------------------------------------------------------------------

# Overwrite - Replaces all existing data
df.write.mode("overwrite").saveAsTable("MY_TABLE")
#   - Use when you want a fresh snapshot
#   - ⚠️  Destructive - existing data is lost

# Append - Adds new rows to existing data
df.write.mode("append").saveAsTable("MY_TABLE")
#   - Use for incremental loads
#   - Beware of duplicates if re-running

# Ignore - Skips write if destination exists
df.write.mode("ignore").saveAsTable("MY_TABLE")
#   - Use for idempotent operations
#   - No error, but data not written

# Error (default) - Fails if destination exists
df.write.mode("error").saveAsTable("MY_TABLE")
#   - Safest option to prevent accidental overwrites
#   - Must explicitly choose overwrite/append

# =============================================================================


## Data Transformations


### Feature Support Matrix:

Understanding what PySpark features are supported helps you write efficient code.

#### ✅ Fully Supported DataFrame Operations:
- `select`, `filter`, `where`
- `groupBy`, `agg` (all aggregation functions)
- `join` (inner, left, right, outer, broadcast)
- `orderBy`, `sort`
- `distinct`, `dropDuplicates`
- Window functions (`row_number`, `rank`, `lag`, `lead`, etc.)
- Built-in functions (95%+ coverage)
- `cache`, `persist` (creates temp tables in Snowflake)

#### ⚠️ Limited Support:
- `repartition` (logical operation only)
- `coalesce` (similar to repartition)
- Python UDFs (work but slow - avoid if possible)
- Pandas UDFs (work but slow - avoid if possible)
- MLlib (partial - transformers work, estimators limited)

#### ❌ NOT Supported:
- RDD API completely
- `.rdd`, `.foreach()`, `.foreachPartition()`
- Structured Streaming
- GraphX
- Custom data sources
- `.checkpoint()`


### Data Types Support

**✅ Supported:**
- String, Integer, Long, Float, Double, Decimal
- Boolean, Date, Timestamp
- Array, Map, Struct
- Binary

**❌ Not Supported:**
- DayTimeIntervalType
- YearMonthIntervalType
- UserDefinedTypes

#### Supported File Formats:
- ✅ Parquet, CSV, JSON, Avro, ORC
- ❌ Delta Lake, Hudi not supported


In [ ]:
# =============================================================================
# DATA TRANSFORMATIONS
# =============================================================================
#
# All transformations push down to Snowflake SQL - no data leaves the warehouse
# until you explicitly collect results
#
# =============================================================================

from pyspark.sql.functions import (
    col, lit, when, coalesce,
    sum, avg, count, min, max, countDistinct, collect_list,
    year, month, dayofmonth, hour, minute, second, dayofweek, dayofyear, weekofyear, quarter,
    to_date, to_timestamp, date_add, date_sub, months_between, datediff, date_trunc,
    upper, lower, initcap, trim, ltrim, rtrim, lpad, rpad,
    concat, substring, regexp_replace, regexp_extract,
    row_number, rank, dense_rank, percent_rank, lag, lead, first, last,
    broadcast
)
from pyspark.sql.window import Window

# -----------------------------------------------------------------------------
# 1. SELECTING AND FILTERING
# -----------------------------------------------------------------------------

# Select specific columns from a DataFrame
df_selected = df.select("id", "name", "amount")
df_selected = df.select(col("id"), col("name").alias("customer_name"))  # with rename

# Filter rows based on conditions
df_filtered = df.filter(col("status") == "active")
df_filtered = df.where(col("amount") > 100)
df_filtered = df.filter((col("status") == "active") & (col("amount") > 100))  # AND
df_filtered = df.filter((col("status") == "active") | (col("amount") > 1000)) # OR

# Remove duplicate rows
df_unique = df.distinct()                                    # all columns
df_unique = df.dropDuplicates(["customer_id", "order_date"]) # specific columns

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 2. ADDING AND MODIFYING COLUMNS
# -----------------------------------------------------------------------------

# Add a new column with calculated values
df = df.withColumn("total", col("price") * col("quantity"))
df = df.withColumn("discounted_price", col("price") * 0.9)
df = df.withColumn("status_flag", when(col("status") == "active", 1).otherwise(0))

# Rename existing columns
df = df.withColumnRenamed("old_name", "new_name")
df = df.toDF("col1", "col2", "col3")  # rename all columns at once

# Cast column to different data type
df = df.withColumn("amount", col("amount").cast("double"))
df = df.withColumn("event_date", col("timestamp_col").cast("date"))

# Replace null values with defaults
df = df.fillna(0, subset=["amount"])                    # single column
df = df.fillna({"amount": 0, "name": "Unknown"})        # multiple columns
df = df.withColumn("value", coalesce(col("value"), lit(0)))  # using coalesce

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 3. AGGREGATIONS
# -----------------------------------------------------------------------------

# Group rows by one or more columns
df_grouped = df.groupBy("category")
df_grouped = df.groupBy("category", "region")

# Apply aggregate functions to groups
df_agg = df.groupBy("category").agg(
    count("*").alias("row_count"),
    countDistinct("customer_id").alias("unique_customers"),
    sum("amount").alias("total_amount"),
    avg("amount").alias("avg_amount"),
    min("amount").alias("min_amount"),
    max("amount").alias("max_amount"),
    collect_list("product_name").alias("products")
)

# Rename aggregated columns with alias
df_agg = df.groupBy("category").agg(
    sum("amount").alias("total_sales"),
    avg("amount").alias("average_sale")
)

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 4. JOINS
# -----------------------------------------------------------------------------

# Join two DataFrames on matching columns
df_joined = df1.join(df2, df1["id"] == df2["id"], "inner")   # Inner join
df_joined = df1.join(df2, "id", "inner")                      # Simpler syntax when column names match
df_joined = df1.join(df2, "customer_id", "left")              # Left join
df_joined = df1.join(df2, "customer_id", "right")             # Right join
df_joined = df1.join(df2, "customer_id", "outer")             # Full outer join
df_joined = df1.crossJoin(df2)                                # Cross join (cartesian)

# Broadcast small tables for faster joins
df_joined = df_large.join(broadcast(df_small), "key_column", "left")

# Handle column name conflicts after join
df_joined = df1.alias("a").join(df2.alias("b"), col("a.id") == col("b.id"))
df_result = df_joined.select(col("a.id"), col("a.name"), col("b.value"))

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 5. WINDOW FUNCTIONS
# -----------------------------------------------------------------------------

# Define a window specification
window_spec = Window.partitionBy("customer_id").orderBy("order_date")
window_spec_frame = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(-2, 0)

# Ranking functions
df = df.withColumn("row_num", row_number().over(window_spec))
df = df.withColumn("rank", rank().over(window_spec))              # gaps for ties
df = df.withColumn("dense_rank", dense_rank().over(window_spec))  # no gaps
df = df.withColumn("pct_rank", percent_rank().over(window_spec))

# Analytic functions
df = df.withColumn("prev_amount", lag("amount", 1).over(window_spec))
df = df.withColumn("next_amount", lead("amount", 1).over(window_spec))
df = df.withColumn("first_order", first("amount").over(window_spec))
df = df.withColumn("last_order", last("amount").over(window_spec))

# Running calculations
window_running = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(Window.unboundedPreceding, 0)
df = df.withColumn("running_total", sum("amount").over(window_running))
df = df.withColumn("running_avg", avg("amount").over(window_running))
df = df.withColumn("running_min", min("amount").over(window_running))
df = df.withColumn("running_max", max("amount").over(window_running))

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 6. DATE AND TIME OPERATIONS
# -----------------------------------------------------------------------------

# Extract components from timestamps
df = df.withColumn("year", year("timestamp_col"))
df = df.withColumn("month", month("timestamp_col"))
df = df.withColumn("day", dayofmonth("timestamp_col"))
df = df.withColumn("hour", hour("timestamp_col"))
df = df.withColumn("minute", minute("timestamp_col"))
df = df.withColumn("second", second("timestamp_col"))
df = df.withColumn("day_of_week", dayofweek("timestamp_col"))
df = df.withColumn("day_of_year", dayofyear("timestamp_col"))
df = df.withColumn("week", weekofyear("timestamp_col"))
df = df.withColumn("quarter", quarter("timestamp_col"))

# Convert strings to dates/timestamps
df = df.withColumn("date_col", to_date("date_string", "yyyy-MM-dd"))
df = df.withColumn("ts_col", to_timestamp("ts_string", "yyyy-MM-dd HH:mm:ss"))

# Date arithmetic
df = df.withColumn("next_week", date_add("date_col", 7))
df = df.withColumn("last_week", date_sub("date_col", 7))
df = df.withColumn("days_diff", datediff("end_date", "start_date"))
df = df.withColumn("months_diff", months_between("end_date", "start_date"))
df = df.withColumn("month_start", date_trunc("month", "date_col"))
df = df.withColumn("week_start", date_trunc("week", "date_col"))

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 7. STRING OPERATIONS
# -----------------------------------------------------------------------------

# Transform string case
df = df.withColumn("upper_name", upper("name"))
df = df.withColumn("lower_name", lower("name"))
df = df.withColumn("title_name", initcap("name"))

# Pattern matching and replacement
df = df.withColumn("has_email", col("text").contains("@"))
df = df.withColumn("cleaned", regexp_replace("text", "[^a-zA-Z0-9]", ""))
df = df.withColumn("domain", regexp_extract("email", "@(.+)$", 1))

# Trim and pad strings
df = df.withColumn("trimmed", trim("text"))
df = df.withColumn("left_trimmed", ltrim("text"))
df = df.withColumn("right_trimmed", rtrim("text"))
df = df.withColumn("padded_id", lpad("id", 10, "0"))    # "42" -> "0000000042"
df = df.withColumn("padded_right", rpad("code", 5, "X")) # "AB" -> "ABXXX"

# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 8. SORTING AND LIMITING
# -----------------------------------------------------------------------------

# Sort results by one or more columns
df_sorted = df.orderBy("name")                              # ascending (default)
df_sorted = df.orderBy(col("amount").desc())                # descending
df_sorted = df.orderBy(col("category").asc(), col("amount").desc())  # multiple columns
df_sorted = df.orderBy(col("value").asc_nulls_first())      # nulls first
df_sorted = df.orderBy(col("value").desc_nulls_last())      # nulls last

# Limit number of rows returned
df_limited = df.limit(10)                # first 10 rows
df_top_10 = df.orderBy(col("amount").desc()).limit(10)  # top 10 by amount

# =============================================================================


In [ ]:
# =============================================================================
# END-TO-END DATA PIPELINE
# =============================================================================
#
# A production-ready pipeline structure with monitoring, error handling,
# and best practices integrated throughout
#
# =============================================================================

# -----------------------------------------------------------------------------
# PIPELINE STRUCTURE OVERVIEW
# -----------------------------------------------------------------------------
#
# 1. Setup & Configuration
# 2. Telemetry Initialization
# 3. Data Ingestion 
# 4. Data Validation
# 5. Transformations 
# 6. Data Quality Checks
# 7. Write Output 
# 8. Cleanup & Summary
#
# -----------------------------------------------------------------------------

# =============================================================================
# STEP 1: SETUP & CONFIGURATION
# =============================================================================

# --- Import required packages ---
import os
import uuid
import logging
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Optional, Dict, Any, List

from snowflake import snowpark_connect
from snowflake.snowpark.context import get_active_session

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import (
    StructType, StructField, 
    StringType, IntegerType, LongType, DoubleType, 
    TimestampType, DateType, BooleanType
)
from pyspark.sql.functions import (
    col, lit, when, coalesce, trim, lower, upper,
    sum, avg, count, min, max, countDistinct,
    year, month, dayofmonth, to_date, current_timestamp,
    row_number, broadcast
)
from pyspark.sql.window import Window

# --- Configuration dataclass for type safety and clarity ---
@dataclass
class PipelineConfig:
    """Centralized configuration for the data pipeline."""
    
    # Pipeline metadata
    pipeline_name: str = "sales_analytics_pipeline"
    pipeline_version: str = "1.0.0"
    environment: str = os.getenv("ENVIRONMENT", "dev")
    
    # Source configuration
    source_table: str = "RAW_DATA.SALES.TRANSACTIONS"
    dimension_table: str = "RAW_DATA.SALES.CUSTOMERS"
    
    # Output configuration
    output_table: str = "ANALYTICS.SALES.DAILY_SUMMARY"
    output_mode: str = "overwrite"
    
    # Processing parameters
    lookback_days: int = 7
    min_expected_rows: int = 1000
    max_expected_rows: int = 10_000_000
    
    # Quality thresholds
    max_null_percentage: float = 0.05  # 5% max nulls allowed
    row_count_variance_threshold: float = 0.20  # 20% variance allowed

config = PipelineConfig()

# --- Initialize Snowpark Connect session ---
session = get_active_session()
spark = snowpark_connect.server.init_spark_session()

print(f"Pipeline: {config.pipeline_name} v{config.pipeline_version}")
print(f"Environment: {config.environment}")

# =============================================================================

# =============================================================================
# STEP 2: TELEMETRY INITIALIZATION
# =============================================================================

# --- Configure logging ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(config.pipeline_name)

# --- Pipeline run tracking ---
@dataclass
class PipelineMetrics:
    """Track metrics throughout pipeline execution."""
    run_id: str
    start_time: datetime
    end_time: Optional[datetime] = None
    status: str = "running"
    
    # Stage metrics
    rows_ingested: int = 0
    rows_after_validation: int = 0
    rows_after_transform: int = 0
    rows_written: int = 0
    
    # Quality metrics
    validation_passed: bool = False
    quality_score: float = 0.0
    warnings: List[str] = None
    
    def __post_init__(self):
        self.warnings = self.warnings or []
    
    def add_warning(self, message: str):
        self.warnings.append(message)
        logger.warning(message)
    
    def duration_seconds(self) -> float:
        end = self.end_time or datetime.now()
        return (end - self.start_time).total_seconds()

# Initialize metrics for this run
metrics = PipelineMetrics(
    run_id=str(uuid.uuid4())[:8],
    start_time=datetime.now()
)

logger.info(f"{'='*60}")
logger.info(f"PIPELINE START | Run ID: {metrics.run_id}")
logger.info(f"{'='*60}")

# =============================================================================

# =============================================================================
# STEP 3: DATA INGESTION
# =============================================================================

def ingest_data(spark: SparkSession, config: PipelineConfig) -> DataFrame:
    """
    Ingest source data with early filtering to minimize data movement.
    
    Best practices applied:
    - Filter at read time to reduce data volume
    - Select only required columns
    - Use Snowflake tables for fastest path
    """
    logger.info(f"Ingesting data from: {config.source_table}")
    
    # Calculate date range for filtering
    end_date = datetime.now()
    start_date = end_date - timedelta(days=config.lookback_days)
    
    # Read with predicate pushdown - filter happens in Snowflake
    df = spark.sql(f"""
        SELECT 
            transaction_id,
            customer_id,
            product_id,
            transaction_date,
            quantity,
            unit_price,
            discount_pct,
            region,
            channel,
            created_at
        FROM {config.source_table}
        WHERE transaction_date >= '{start_date.strftime('%Y-%m-%d')}'
          AND transaction_date < '{end_date.strftime('%Y-%m-%d')}'
    """)
    
    return df

# Execute ingestion
logger.info("STEP 3: Data Ingestion")
df_raw = ingest_data(spark, config)
metrics.rows_ingested = df_raw.count()
logger.info(f"Rows ingested: {metrics.rows_ingested:,}")

# =============================================================================

# =============================================================================
# STEP 4: DATA VALIDATION
# =============================================================================

class ValidationError(Exception):
    """Raised when critical validation fails."""
    pass

def validate_dataframe(
    df: DataFrame, 
    config: PipelineConfig,
    metrics: PipelineMetrics
) -> DataFrame:
    """
    Validate source data before processing.
    
    Checks performed:
    - Row count within expected range
    - Required columns present
    - Null percentage within threshold
    - No duplicate primary keys
    """
    logger.info("STEP 4: Data Validation")
    errors = []
    
    # --- Check 1: Row count bounds ---
    row_count = metrics.rows_ingested
    if row_count < config.min_expected_rows:
        errors.append(f"Row count {row_count:,} below minimum {config.min_expected_rows:,}")
    if row_count > config.max_expected_rows:
        errors.append(f"Row count {row_count:,} exceeds maximum {config.max_expected_rows:,}")
    
    # --- Check 2: Required columns present ---
    required_columns = ["transaction_id", "customer_id", "transaction_date", "quantity", "unit_price"]
    missing_columns = [c for c in required_columns if c not in df.columns]
    if missing_columns:
        errors.append(f"Missing required columns: {missing_columns}")
    
    # --- Check 3: Null percentage in key columns ---
    key_columns = ["transaction_id", "customer_id", "quantity", "unit_price"]
    for col_name in key_columns:
        if col_name in df.columns:
            null_count = df.filter(col(col_name).isNull()).count()
            null_pct = null_count / row_count if row_count > 0 else 0
            if null_pct > config.max_null_percentage:
                errors.append(f"Column '{col_name}' has {null_pct:.1%} nulls (max: {config.max_null_percentage:.1%})")
            elif null_pct > 0:
                metrics.add_warning(f"Column '{col_name}' has {null_pct:.1%} nulls")
    
    # --- Check 4: No duplicate primary keys ---
    pk_count = df.select("transaction_id").distinct().count()
    if pk_count < row_count:
        duplicate_count = row_count - pk_count
        errors.append(f"Found {duplicate_count:,} duplicate transaction_ids")
    
    # --- Handle validation results ---
    if errors:
        for error in errors:
            logger.error(f"VALIDATION FAILED: {error}")
        raise ValidationError(f"Validation failed with {len(errors)} error(s)")
    
    metrics.validation_passed = True
    metrics.rows_after_validation = row_count
    logger.info("Validation PASSED")
    
    return df

# Execute validation
df_validated = validate_dataframe(df_raw, config, metrics)

# =============================================================================

# =============================================================================
# STEP 5: TRANSFORMATIONS
# =============================================================================

def transform_data(
    df: DataFrame,
    df_customers: DataFrame,
    metrics: PipelineMetrics
) -> DataFrame:
    """
    Apply business transformations to the validated data.
    
    Stages:
    5A. Data Cleaning
    5B. Business Logic  
    5C. Joins & Enrichment
    5D. Aggregations
    """
    logger.info("STEP 5: Transformations")
    
    # -------------------------------------------------------------------------
    # STAGE 5A: Data Cleaning
    # -------------------------------------------------------------------------
    logger.info("  5A: Data Cleaning")
    
    df_clean = df \
        .withColumn("region", trim(upper(col("region")))) \
        .withColumn("channel", trim(lower(col("channel")))) \
        .withColumn("discount_pct", coalesce(col("discount_pct"), lit(0.0))) \
        .filter(col("quantity") > 0) \
        .filter(col("unit_price") > 0)
    
    # -------------------------------------------------------------------------
    # STAGE 5B: Business Logic
    # -------------------------------------------------------------------------
    logger.info("  5B: Business Logic")
    
    df_enriched = df_clean \
        .withColumn(
            "gross_amount", 
            col("quantity") * col("unit_price")
        ) \
        .withColumn(
            "discount_amount",
            col("gross_amount") * col("discount_pct")
        ) \
        .withColumn(
            "net_amount",
            col("gross_amount") - col("discount_amount")
        ) \
        .withColumn(
            "transaction_year",
            year("transaction_date")
        ) \
        .withColumn(
            "transaction_month",
            month("transaction_date")
        ) \
        .withColumn(
            "size_category",
            when(col("net_amount") >= 1000, "large")
            .when(col("net_amount") >= 100, "medium")
            .otherwise("small")
        )
    
    # -------------------------------------------------------------------------
    # STAGE 5C: Joins & Enrichment
    # -------------------------------------------------------------------------
    logger.info("  5C: Joins & Enrichment")
    
    # Broadcast small dimension table for faster join
    df_with_customer = df_enriched.join(
        broadcast(df_customers.select(
            col("customer_id"),
            col("customer_segment"),
            col("customer_tier"),
            col("acquisition_date")
        )),
        on="customer_id",
        how="left"
    )
    
    # Log join quality
    total_rows = df_enriched.count()
    matched_rows = df_with_customer.filter(col("customer_segment").isNotNull()).count()
    match_rate = matched_rows / total_rows if total_rows > 0 else 0
    logger.info(f"  Customer join match rate: {match_rate:.1%}")
    
    if match_rate < 0.95:
        metrics.add_warning(f"Customer join match rate below 95%: {match_rate:.1%}")
    
    # -------------------------------------------------------------------------
    # STAGE 5D: Aggregations
    # -------------------------------------------------------------------------
    logger.info("  5D: Aggregations")
    
    df_aggregated = df_with_customer.groupBy(
        "transaction_date",
        "region",
        "channel",
        "customer_segment",
        "size_category"
    ).agg(
        count("*").alias("transaction_count"),
        countDistinct("customer_id").alias("unique_customers"),
        sum("quantity").alias("total_quantity"),
        sum("gross_amount").alias("total_gross"),
        sum("discount_amount").alias("total_discount"),
        sum("net_amount").alias("total_net"),
        avg("net_amount").alias("avg_transaction_value"),
        min("net_amount").alias("min_transaction_value"),
        max("net_amount").alias("max_transaction_value")
    )
    
    # Add metadata columns
    df_final = df_aggregated \
        .withColumn("pipeline_run_id", lit(metrics.run_id)) \
        .withColumn("processed_at", current_timestamp())
    
    metrics.rows_after_transform = df_final.count()
    logger.info(f"  Rows after aggregation: {metrics.rows_after_transform:,}")
    
    return df_final

# Load dimension table
df_customers = spark.read.table(config.dimension_table)

# Execute transformations
df_transformed = transform_data(df_validated, df_customers, metrics)

# Cache if we need to access multiple times (quality checks + write)
df_transformed.cache()

# =============================================================================

# =============================================================================
# STEP 6: DATA QUALITY CHECKS
# =============================================================================

def quality_checks(
    df: DataFrame,
    metrics: PipelineMetrics
) -> float:
    """
    Validate output data quality before writing.
    
    Returns quality score (0.0 - 1.0).
    """
    logger.info("STEP 6: Data Quality Checks")
    checks_passed = 0
    total_checks = 4
    
    row_count = metrics.rows_after_transform
    
    # --- Check 1: Non-zero output ---
    if row_count > 0:
        checks_passed += 1
        logger.info("  ✓ Output has rows")
    else:
        logger.error("  ✗ Output is empty!")
    
    # --- Check 2: No duplicate keys ---
    key_columns = ["transaction_date", "region", "channel", "customer_segment", "size_category"]
    distinct_keys = df.select(key_columns).distinct().count()
    if distinct_keys == row_count:
        checks_passed += 1
        logger.info("  ✓ No duplicate keys")
    else:
        logger.error(f"  ✗ Found {row_count - distinct_keys:,} duplicate keys")
    
    # --- Check 3: Amounts are positive ---
    negative_amounts = df.filter(col("total_net") < 0).count()
    if negative_amounts == 0:
        checks_passed += 1
        logger.info("  ✓ All amounts positive")
    else:
        metrics.add_warning(f"Found {negative_amounts:,} rows with negative amounts")
    
    # --- Check 4: Transaction counts make sense ---
    total_transactions = df.agg(sum("transaction_count")).collect()[0][0]
    if total_transactions >= metrics.rows_after_validation * 0.9:
        checks_passed += 1
        logger.info("  ✓ Transaction counts reconcile")
    else:
        metrics.add_warning("Transaction count reconciliation variance > 10%")
    
    quality_score = checks_passed / total_checks
    logger.info(f"  Quality Score: {quality_score:.0%} ({checks_passed}/{total_checks} checks passed)")
    
    return quality_score

# Execute quality checks
metrics.quality_score = quality_checks(df_transformed, metrics)

# =============================================================================

# =============================================================================
# STEP 7: WRITE OUTPUT
# =============================================================================

def write_output(
    df: DataFrame,
    config: PipelineConfig,
    metrics: PipelineMetrics
) -> int:
    """
    Write transformed data to destination table.
    
    Best practices:
    - Use Snowflake tables for fastest write path
    - Verify row count after write
    """
    logger.info("STEP 7: Write Output")
    logger.info(f"  Destination: {config.output_table}")
    logger.info(f"  Mode: {config.output_mode}")
    
    # Write to Snowflake table
    df.write \
        .mode(config.output_mode) \
        .saveAsTable(config.output_table)
    
    # Verify write success
    written_count = spark.read.table(config.output_table).count()
    logger.info(f"  Rows written: {written_count:,}")
    
    return written_count

# Execute write
metrics.rows_written = write_output(df_transformed, config, metrics)

# =============================================================================

# =============================================================================
# STEP 8: CLEANUP & SUMMARY
# =============================================================================

def cleanup_and_summarize(
    df_cached: DataFrame,
    metrics: PipelineMetrics
):
    """
    Release resources and log final summary.
    
    Always runs, even on failure (call in finally block).
    """
    logger.info("STEP 8: Cleanup & Summary")
    
    # --- Release cached DataFrames ---
    try:
        df_cached.unpersist()
        logger.info("  Cached DataFrames released")
    except Exception as e:
        logger.warning(f"  Could not unpersist DataFrame: {e}")
    
    # --- Finalize metrics ---
    metrics.end_time = datetime.now()
    metrics.status = "success" if metrics.quality_score >= 0.75 else "completed_with_warnings"
    
    # --- Log summary ---
    logger.info(f"{'='*60}")
    logger.info("PIPELINE SUMMARY")
    logger.info(f"{'='*60}")
    logger.info(f"  Run ID:           {metrics.run_id}")
    logger.info(f"  Status:           {metrics.status.upper()}")
    logger.info(f"  Duration:         {metrics.duration_seconds():.1f} seconds")
    logger.info(f"  Rows In:          {metrics.rows_ingested:,}")
    logger.info(f"  Rows Out:         {metrics.rows_written:,}")
    logger.info(f"  Quality Score:    {metrics.quality_score:.0%}")
    
    if metrics.warnings:
        logger.info(f"  Warnings ({len(metrics.warnings)}):")
        for warning in metrics.warnings:
            logger.info(f"    - {warning}")
    
    logger.info(f"{'='*60}")

# Execute cleanup (in production, wrap main pipeline in try/finally)
cleanup_and_summarize(df_transformed, metrics)

# =============================================================================

# =============================================================================
# BEST PRACTICES CHECKLIST (Applied in this pipeline)
# =============================================================================
#
# ✅ PERFORMANCE:
#    - Filter at read time (predicate pushdown)
#    - Broadcast small dimension tables in joins
#    - Cache DataFrames accessed multiple times
#    - Use built-in SQL functions (no Python UDFs)
#
# ✅ RELIABILITY:
#    - Validate data at input (row counts, nulls, duplicates)
#    - Validate data at output (quality checks)
#    - Use dataclasses for type-safe configuration
#    - Clean up resources (unpersist cached data)
#
# ✅ OBSERVABILITY:
#    - Structured logging with timestamps
#    - Unique run ID for tracing
#    - Row counts logged at each stage
#    - Quality score and warnings tracked
#
# ✅ MAINTAINABILITY:
#    - Configuration separated from logic
#    - Clear stage separation with comments
#    - Functions with docstrings
#    - Consistent naming conventions
#
# =============================================================================

### Performance Considerations

Follow these best practices to get optimal performance from Snowpark Connect.

1. Use SQL Functions Over UDFs: Python UDFs require data to be transferred to the client, processed, and sent back - this is 10-100x slower than native operations!
2. Broadcast Joins for Small Tables: When joining a large table with a small dimension table, use `broadcast()` to optimize the join.
3. Cache Frequently Accessed DataFrames: Caching creates temporary tables in Snowflake for faster repeated access. Remember to `unpersist()` when done!
4. Minimize Data Movement: Process data in Snowflake and only transfer final results. Avoid `collect()` on large datasets!
5. Partition Awareness: Filter on partitioned columns to enable partition pruning and reduce data scanned.

## Additional Resources

**Official Documentation:**
- [Snowflake Documentation](https://docs.snowflake.com/)
- [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/)